# 🌿 Классификация состояния растений — исследование

Ноутбук для анализа данных, визуализации и результатов обучения.

In [ ]:
import os
import json
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from collections import Counter

sns.set_style('whitegrid')
DATA_DIR = 'data/PlantVillage'
print('Готово')

## 1. Обзор датасета

In [ ]:
if os.path.exists(DATA_DIR):
    classes = sorted([d for d in os.listdir(DATA_DIR) if os.path.isdir(os.path.join(DATA_DIR, d))])
    counts = {c: len(os.listdir(os.path.join(DATA_DIR, c))) for c in classes}
    total = sum(counts.values())
    print(f'Классов: {len(classes)}, Изображений: {total}')
    print(f'Мин: {min(counts.values())}, Макс: {max(counts.values())}, Среднее: {np.mean(list(counts.values())):.0f}')
else:
    print(f'Данные не найдены: {DATA_DIR}. Скачайте датасет (см. README).')

## 2. Распределение классов

In [ ]:
if os.path.exists(DATA_DIR):
    fig, ax = plt.subplots(figsize=(12, 9))
    names = [c.replace('___', '\n') for c in counts.keys()]
    vals = list(counts.values())
    colors = ['#2ecc71' if 'healthy' in c.lower() else '#e74c3c' for c in counts.keys()]
    ax.barh(range(len(names)), vals, color=colors)
    ax.set_yticks(range(len(names)))
    ax.set_yticklabels(names, fontsize=7)
    ax.set_xlabel('Количество изображений')
    ax.set_title('Распределение классов PlantVillage', fontweight='bold')
    ax.invert_yaxis()
    from matplotlib.patches import Patch
    ax.legend(handles=[Patch(color='#2ecc71', label='Здоровые'), Patch(color='#e74c3c', label='Больные')])
    plt.tight_layout()
    plt.savefig('results/class_distribution.png', dpi=150)
    plt.show()

## 3. Примеры изображений

In [ ]:
if os.path.exists(DATA_DIR):
    fig, axes = plt.subplots(3, 6, figsize=(18, 9))
    for idx, cls in enumerate(classes[:18]):
        r, c = idx // 6, idx % 6
        img_name = os.listdir(os.path.join(DATA_DIR, cls))[0]
        img = Image.open(os.path.join(DATA_DIR, cls, img_name))
        axes[r][c].imshow(img)
        axes[r][c].set_title(cls.replace('___', '\n'), fontsize=7)
        axes[r][c].axis('off')
    plt.suptitle('Примеры изображений', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.savefig('results/sample_images.png', dpi=150)
    plt.show()

## 4. Примеры аугментации

In [ ]:
from torchvision import transforms
import torch

if os.path.exists(DATA_DIR):
    aug_transform = transforms.Compose([
        transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
        transforms.RandomHorizontalFlip(),
        transforms.RandomRotation(25),
        transforms.ColorJitter(0.2, 0.2, 0.2, 0.1),
        transforms.ToTensor(),
    ])
    
    sample_path = os.path.join(DATA_DIR, classes[0], os.listdir(os.path.join(DATA_DIR, classes[0]))[0])
    orig = Image.open(sample_path).convert('RGB')
    
    fig, axes = plt.subplots(2, 5, figsize=(18, 7))
    axes[0][0].imshow(orig)
    axes[0][0].set_title('Оригинал', fontweight='bold')
    axes[0][0].axis('off')
    for i in range(1, 10):
        r, c = i // 5, i % 5
        aug_img = aug_transform(orig).permute(1, 2, 0).numpy()
        axes[r][c].imshow(aug_img)
        axes[r][c].set_title(f'Аугментация {i}')
        axes[r][c].axis('off')
    plt.suptitle('Примеры аугментации данных', fontweight='bold')
    plt.tight_layout()
    plt.savefig('results/augmentation_examples.png', dpi=150)
    plt.show()

## 5. Результаты обучения

Сначала обучите модель:
```bash
python train.py --data_dir data/PlantVillage --epochs 20
```

In [ ]:
history_path = 'results/training_history.json'
if os.path.exists(history_path):
    with open(history_path) as f:
        h = json.load(f)
    
    ep = range(1, len(h['train_loss']) + 1)
    fig, (a1, a2) = plt.subplots(1, 2, figsize=(14, 5))
    a1.plot(ep, h['train_loss'], 'b-o', label='Train', ms=4)
    a1.plot(ep, h['val_loss'], 'r-o', label='Val', ms=4)
    a1.set(xlabel='Epoch', ylabel='Loss', title='Loss')
    a1.legend(); a1.grid(alpha=0.3)
    
    a2.plot(ep, h['train_acc'], 'b-o', label='Train', ms=4)
    a2.plot(ep, h['val_acc'], 'r-o', label='Val', ms=4)
    a2.set(xlabel='Epoch', ylabel='Accuracy (%)', title='Accuracy')
    a2.legend(); a2.grid(alpha=0.3)
    plt.tight_layout(); plt.show()
    print(f'Best Val Acc: {max(h["val_acc"]):.2f}%')
else:
    print('Обучите модель сначала.')

## 6. Метрики оценки

```bash
python evaluate.py --data_dir data/PlantVillage --model_path results/best_model.pth
```

In [ ]:
metrics_path = 'results/metrics.json'
if os.path.exists(metrics_path):
    with open(metrics_path) as f:
        m = json.load(f)
    print('=== Метрики на тесте ===')
    for k, v in m.items():
        if isinstance(v, float):
            print(f'  {k}: {v:.4f}')
        elif isinstance(v, int):
            print(f'  {k}: {v}')
    
    if 'per_class_accuracy' in m:
        pca = m['per_class_accuracy']
        sorted_pca = sorted(pca.items(), key=lambda x: x[1])
        print('\nХудшие 5:')
        for cls, acc in sorted_pca[:5]:
            print(f'  {cls}: {acc:.4f}')
        print('\nЛучшие 5:')
        for cls, acc in sorted_pca[-5:]:
            print(f'  {cls}: {acc:.4f}')
else:
    print('Запустите evaluate.py сначала.')

In [ ]:
cm_path = 'results/confusion_matrix.png'
if os.path.exists(cm_path):
    img = Image.open(cm_path)
    plt.figure(figsize=(14, 14))
    plt.imshow(img); plt.axis('off')
    plt.title('Confusion Matrix', fontsize=14, fontweight='bold')
    plt.show()
else:
    print('Запустите evaluate.py сначала.')

## 7. Выводы

1. **EfficientNet-B0** с transfer learning достигает ~96-98% accuracy на PlantVillage
2. **Аугментация** предотвращает переобучение
3. **Заморозка backbone** на первых эпохах стабилизирует обучение
4. Некоторые визуально похожие болезни путаются (early/late blight)

### Возможные улучшения
- Ансамбль моделей
- Test-time augmentation
- GradCAM для интерпретируемости
- Дообучение на реальных снимках с дронов